# DSC 80 - Discussion 06

### Due Saturday Nov 5th, 11:59:59PM


# Web Scraping

In [1]:
import os
import numpy as np
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import re

In [2]:
# from discussion import *

In [3]:
import os
import numpy as np
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import re

## Let's scrape some books data!
* We will be scraping some book information from http://books.toscrape.com/index.html
* Note that this website is built for scraping purposes

### 1. HTML

* **Hyper Text Markup Language** is used to provide content to a web page and instruct web browsers (like Chrome, Safari etc.) on how to structure that content
* HTML tags form the builing blocks of structuring a webpage - [HTML Tags Reference](https://www.w3schools.com/tags/default.asp)
* **CSS** is used to style the web pages and can be used inplace inside the HTML code or by using external css styling files
* **JavaScript** is used to provide interactivity within the browser


* Note that you don't need be a web designer or developer to scrape data from standard websites. Tools like BeautifulSoup with basic knowledge of HTML tags will provide a great starting point

## HTML Pages are DOM types
<img src = 'imgs/dom.jpg'></img>

### 2. Get all the book urls from single catalogue page

In [4]:
url = 'http://books.toscrape.com/catalogue/page-1.html'

In [5]:
book_request = requests.get(url)
soup = BeautifulSoup(book_request.text)
soup

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:30" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="../static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="../static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link href="

The key to scraping is in identifying the container/block/division that is unique to your requirement. Also note that there might exist multiple ways to scrape the same information.

<img src='imgs/scraping_container.png'>

In [6]:
articles = soup.find_all('article', {"class": "product_pod"})
articles

[<article class="product_pod">
 <div class="image_container">
 <a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
 </div>
 <p class="star-rating Three">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>
 <h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
 <div class="product_price">
 <p class="price_color">Â£51.77</p>
 <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>
 <form>
 <button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
 </form>
 </div>
 </article>,
 <article class="product_pod">
 <div class="image_container">
 <a href="tipping-the-velvet_999/index.html"><img alt="Tipping the Velvet" class="thumbnail" src="../media/cach

In [28]:
articles[0].find('a')

<a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>

In [8]:
book_links = [art.find('a').get('href') for art in articles]
book_links

['a-light-in-the-attic_1000/index.html',
 'tipping-the-velvet_999/index.html',
 'soumission_998/index.html',
 'sharp-objects_997/index.html',
 'sapiens-a-brief-history-of-humankind_996/index.html',
 'the-requiem-red_995/index.html',
 'the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
 'the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
 'the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html',
 'the-black-maria_991/index.html',
 'starving-hearts-triangular-trade-trilogy-1_990/index.html',
 'shakespeares-sonnets_989/index.html',
 'set-me-free_988/index.html',
 'scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html',
 'rip-it-up-and-start-again_986/index.html',
 'our-band-could-be-your-life-scenes-from-the-american-indie-underground-1981-1991_985/index.html',
 'olio_984/index.html',
 'mesaerion-the-best-science-fiction-stories-1800-1849_983/index.html',

### 3. Parse Book information from a book URL

Let's try and extract the title, price, instock availability and rating of the book.

In [9]:
# For the first book
book_url = 'http://books.toscrape.com/catalogue/' + book_links[0]

In [10]:
book_soup = BeautifulSoup(requests.get(book_url).text)
book_soup

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    A Light in the Attic | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="
    It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now

#### 3.1 Book Title

In [11]:
book_soup.find_all('h1')
# book_soup.find_all('h2')

[<h1>A Light in the Attic</h1>]

In [12]:
name = book_soup.find('article', {'class':'product_page'}).find('h1').text
# name = book_soup.find('article', {'class':'product_page'}).h1.text
name

'A Light in the Attic'

#### 3.2 Book Price

In [13]:
price = book_soup.find('p', {'class':'price_color'}).text
price

'Â£51.77'

In [14]:
price = float(price[2:])
price

51.77

#### 3.3 Book Availability

In [15]:
instock = book_soup.find('p', {'class':'instock availability'}).text
instock

'\n\n    \n        In stock (22 available)\n    \n'

In [16]:
instock_units = re.findall(r'\d+', instock)
# Note that you can do this using string methods like .isdigit() as well
instock_units

['22']

#### 3.4 Book Rating

In [17]:
rating = book_soup.find('p', class_ = re.compile('star-rating')).get('class')[1]
rating

'Three'

**Question 1**

Get all the catalog urls of the website in the form of a list. Notice that each page of the website has a very specific pattern.

Hint: Try clicking the "next" button at the bottom of the page to see how the URL changes.

In [18]:
def get_website_urls():
    """
    Get all the website URLs

    :Example:
    >>> urls = get_website_urls()
    >>> len(urls)
    50
    >>> 'catalogue' in urls[0]
    True
    """
    # BEGIN SOLUTION
    return [f'http://books.toscrape.com/catalogue/page-{idx}.html' for idx in range(1,51)]
    # END SOLUTION

In [19]:
""" # BEGIN TEST CONFIG
points: 1
failure_message: 'doctest'
""" # END TEST CONFIG
urls = get_website_urls()
len(urls)==50

True

In [20]:
""" # BEGIN TEST CONFIG
points: 2
failure_message: 'sum of all page numbers'
""" # END TEST CONFIG
urls = get_website_urls()
page_nums = [int(re.findall('page-(\d)',url)[0]) for url in urls]
sum(page_nums)==150

True

**Question 2**

Extract all the category names (`Travel`, `Mystery`..) on the home page (http://books.toscrape.com/index.html) and return them as a list. Notice to remove all the extra whitespaces if any.

In [21]:
def book_categories():
    """
    Get all the book categories and return them as a list

    :Example:
    >>> categories = book_categories()
    >>> len(categories)
    50
    >>> 'Classics' in categories
    True
    """
    # BEGIN SOLUTION
    book_request = requests.get('http://books.toscrape.com/index.html')
    soup = BeautifulSoup(book_request.text)
    
    sidebar = soup.find_all('div', {"class": "side_categories"})
    
    result = []
    for anchor in sidebar[0].find_all('a'):
        result.append(anchor.text)
    
    result = [i.strip() for i in result]
    return result[1:]

    # END SOLUTION

In [22]:
# don't change this cell -- it is needed for the tests to work
cats = book_categories()

In [23]:
""" # BEGIN TEST CONFIG
points: 2
failure_message: 'doctest'
""" # END TEST CONFIG
len(cats)==50

True

In [24]:
""" # BEGIN TEST CONFIG
points: 2
failure_message: 'doctest'
""" # END TEST CONFIG
'Classics' in cats

True

In [25]:
""" # BEGIN TEST CONFIG
points: 3
failure_message: 'max and min by alphabet sort'
""" # END TEST CONFIG
min(cats)=='Academic' and max(cats)=='Young Adult'

True

## Congratulations! You're done!

* Submit your `.py` file to Gradescope. Note that you only need to submit the `.py` file; this notebook should not be uploaded. Make sure that all of your work is in the `.py` file and not here by running the doctests: `python -m doctest discussion.py`.